In [197]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import cv2
import numpy as np

In [222]:
batch_size = 256
img_height = 100
img_width = 100

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/Users/danielkashkett/Desktop/EyeProject/data/eye_imgs/',
  validation_split=0.2,
    color_mode="grayscale",
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/Users/danielkashkett/Desktop/EyeProject/data/eye_imgs/',
  validation_split=0.2,
  color_mode="grayscale",
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 20151 files belonging to 5 classes.
Using 16121 files for training.
Found 20151 files belonging to 5 classes.
Using 4030 files for validation.


In [223]:
class_names = train_ds.class_names
print(class_names)

['center', 'down', 'left', 'right', 'up']


In [224]:
num_classes = 5

input_layer = keras.Input(shape=(img_height, img_width, 1))
preprocessing = keras.layers.experimental.preprocessing.Rescaling(1./255)(input_layer)
conv1 = keras.layers.Conv2D(50, 1, padding='same', activation='relu')(preprocessing)
pool1 = keras.layers.MaxPooling2D()(conv1)
conv2 = keras.layers.Conv2D(100, 1, padding='same', activation='relu')(pool1)
pool2 = keras.layers.MaxPooling2D()(conv2)
conv3 = layers.Conv2D(200, 1, padding='same', activation='relu')(pool2)
pool3 = keras.layers.MaxPooling2D()(conv3)
dropout = keras.layers.Dropout(0.2)(pool3)
flatten = keras.layers.Flatten()(dropout)
dense = keras.layers.Dense(128, activation='relu')(flatten)
output_layer = keras.layers.Dense(num_classes,activation='softmax')(dense)

model = keras.Model(inputs=input_layer, outputs=output_layer, name="jupiter3")

# opt = tf.keras.optimizers.Adam(learning_rate=0.003, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
opt = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9)
# model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
model.compile(optimizer=opt, 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# for i in range(0,10):
#     model.layers[i].trainable = False
    
model.summary()

Model: "jupiter3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        [(None, 100, 100, 1)]     0         
_________________________________________________________________
rescaling_22 (Rescaling)     (None, 100, 100, 1)       0         
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 100, 100, 50)      100       
_________________________________________________________________
max_pooling2d_66 (MaxPooling (None, 50, 50, 50)        0         
_________________________________________________________________
conv2d_76 (Conv2D)           (None, 50, 50, 100)       5100      
_________________________________________________________________
max_pooling2d_67 (MaxPooling (None, 25, 25, 100)       0         
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 25, 25, 200)       202

In [225]:
# num_classes = 5

# input_layer = keras.Input(shape=(img_height, img_width, 1))
# preprocessing = keras.layers.experimental.preprocessing.Rescaling(1./255)(input_layer)
# conv1 = keras.layers.Conv2D(16, 1, padding='same', activation='relu')(preprocessing)
# pool1 = keras.layers.MaxPooling2D()(conv1)
# conv2 = keras.layers.Conv2D(32, 1, padding='same', activation='relu')(pool1)
# pool2 = keras.layers.MaxPooling2D()(conv2)
# conv3 = layers.Conv2D(64, 1, padding='same', activation='relu')(pool2)
# pool3 = keras.layers.MaxPooling2D()(conv3)
# dropout = keras.layers.Dropout(0.2)(pool3)
# flatten = keras.layers.Flatten()(dropout)
# dense = keras.layers.Dense(128, activation='relu')(flatten)
# output_layer = keras.layers.Dense(num_classes,activation='softmax')(dense)

# model = keras.Model(inputs=input_layer, outputs=output_layer, name="jupiter3")

# # opt = tf.keras.optimizers.Adam(learning_rate=0.003, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
# opt = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9)
# # model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
# model.compile(optimizer=opt, 
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

# # for i in range(0,10):
# #     model.layers[i].trainable = False
    
# model.summary()

In [226]:
epochs = 2
model.fit(
  train_ds,
  validation_data=val_ds,
    epochs=epochs,
    batch_size=batch_size,
)

Epoch 1/2
63/63 [==============================] - 88s 1s/step - loss: 1.6079 - accuracy: 0.2738 - val_loss: 1.6050 - val_accuracy: 0.3340
Epoch 2/2
63/63 [==============================] - 93s 1s/step - loss: 1.6033 - accuracy: 0.3168 - val_loss: 1.5994 - val_accuracy: 0.3340


In [97]:
# model.save('./Models/jupiter2')

INFO:tensorflow:Assets written to: ./Models/jupiter2/assets


In [193]:
newmodel = tf.keras.models.load_model('../models/jupiter2/')

In [180]:
img1 = cv2.imread('./a22.jpg',cv2.IMREAD_UNCHANGED)
img2 = cv2.imread('./a22.jpg',cv2.IMREAD_UNCHANGED)
# img2 = cv2.imread('../data/eye_imgs/left/a10.jpg' ,cv2.IMREAD_UNCHANGED)
# img1 = img1.reshape((100,100,1))
# img2 = img2.reshape((100,100,1))
img1 = np.expand_dims(img1,0)
img2 = np.expand_dims(img2,0)

In [181]:
img1.shape

(1, 100, 100)

In [182]:
eyes = np.concatenate((img1,img2))

In [183]:
eyes.shape

(2, 100, 100)

In [184]:
preds = model.predict(eyes)

In [195]:
newmodel.predict(img1).max()

7.1146994

In [75]:
preds.shape

(2, 5)

In [77]:
np.random.rand(2,5)

array([[0.19245677, 0.08666217, 0.63213127, 0.61192004, 0.26561435],
       [0.08066118, 0.13698925, 0.67381752, 0.48385871, 0.65951644]])

In [37]:
preds2 = preds1

In [107]:
class PredictionWindow(object):
    
    def __init__(self, size=5):
        self.size = size
        self.items = []
    
    def insert_prediction(self, prediction):
        if len(self.items) == self.size:
            self.items.insert(0,prediction)
            self.items.pop()
        else:
            self.items.append(prediction)
            
    def insert_predictions(self, predictions):
        if (len(self.items) == self.size):
            self.items.insert(0,predictions[0])
            self.items.insert(1,predictions[1])
            self.items.pop()
            self.items.pop() 
        elif (len(self.items) == self.size -1):
            self.items.insert(0,predictions[0])
            self.items.insert(1,predictions[1])
            self.items.pop()
        else:
            self.items.extend(predictions)
    
    def predict(self):
        window = np.array(self.items)
        mean_scores = np.mean(window,axis=0)
        pred = mean_scores.argmax()
        return pred


In [122]:
window = PredictionWindow(size =6)

In [123]:
preds = list(preds)

In [124]:
window.insert_predictions(np.random.rand(2,5))

In [125]:
window.items

[array([0.91951894, 0.30401413, 0.01473367, 0.58865404, 0.60164443]),
 array([0.96556152, 0.75110749, 0.06155948, 0.65131349, 0.03253258])]

In [126]:
window.insert_predictions(np.random.rand(2,5))
window.items

[array([0.91951894, 0.30401413, 0.01473367, 0.58865404, 0.60164443]),
 array([0.96556152, 0.75110749, 0.06155948, 0.65131349, 0.03253258]),
 array([0.56924852, 0.9064319 , 0.56479478, 0.10084006, 0.83186539]),
 array([0.14850993, 0.72629412, 0.71581791, 0.75950631, 0.61835229])]

In [127]:
window.insert_predictions(np.random.rand(2,5))
window.items

[array([0.91951894, 0.30401413, 0.01473367, 0.58865404, 0.60164443]),
 array([0.96556152, 0.75110749, 0.06155948, 0.65131349, 0.03253258]),
 array([0.56924852, 0.9064319 , 0.56479478, 0.10084006, 0.83186539]),
 array([0.14850993, 0.72629412, 0.71581791, 0.75950631, 0.61835229]),
 array([0.24325409, 0.48779145, 0.68789064, 0.50463723, 0.79567773]),
 array([0.72827558, 0.47268944, 0.15936592, 0.39766187, 0.90709574])]

In [128]:
window.predict()

4

In [94]:
np.mean(items,axis=0)

array([0.70270896, 0.58450846, 0.6838695 , 0.55168465, 0.5460241 ])

In [91]:
(0.26226795 + 0.99133883 + 0.85452011)/3

0.7027089633333334

In [45]:
np.concatenate([arr,preds1])

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

In [41]:
preds2.shape

(2, 5)

In [40]:
np.concatenate([preds1,preds2]).shape

(4, 5)

In [125]:
np.argmax(model.predict(img1))

0

In [119]:
arr = np.array([1,2,3,4,5,1])

In [124]:
from collections import Counter
Counter(arr).most_common(1)[0][0]

1

In [24]:
def joint_prediction():
    pass

In [42]:

img2 = np.expand_dims(img2,0)

In [43]:
a = np.concatenate((img1,img2))

In [27]:
a.shape

(2, 100, 100)

In [45]:
model.predict(a)

array([ 2.0141094, -0.907617 ,  3.9984462, -2.9328032,  1.9947188],
      dtype=float32)

In [53]:
def predict_both(left, right):
    img1 = np.expand_dims(left,0)
    img2 = np.expand_dims(right,0)
    concat = np.concatenate((img1,img2))
    classes = model.predict_classes(concat)
    if classes[0] == classes[1]:
        return classes[0]
    else:
        pred = model.predict(concat)
        a = (pred[0].argmax(), pred[0].max())
        b = (pred[1].argmax(), pred[1].max())
        if a[1] > b[1]:
            return a[0]
        else:
            return b[0]

In [ ]:
def predict_frame(self, eye_img):
    img = eye_img.copy()
    batch = np.expand_dims(img,0)
    prediction = self.model.predict_classes(batch)[0]
    return prediction

In [211]:
class_names

['center', 'down', 'left', 'right', 'up']

In [212]:
model.predict_classes(img)[0]

2